In [1]:
import pandas
import datetime
import time
# import the dataset creation module
import os
import sys
module_path = os.path.abspath(os.path.join('../data_pipeline/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import create_onset_data
import omn_utils
import batch_utils

/usr/local/lib/python2.7/dist-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/dist-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/dist-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/usr/local/lib/python2.7/dist-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/usr/local/lib/python2.7/dist-packages/pandas/core/indexes/base.py:7: RuntimeWarning: numpy.dtype size changed, may indicate bi

In [2]:
# dataObj = create_onset_data.OnsetData()

In [3]:
# print len(dataObj.polarDateList), len(dataObj.imageDateList)

In [4]:
# x = time.time()
# finDF = dataObj.create_output_bins()
# y = time.time()
# print y-x

In [5]:
# finDF.head()

In [6]:
# finDF = pandas.read_feather("../data/binned_data.feather")
# finDF.head()

In [7]:
start_date=datetime.datetime(1997,1,1)
end_date=datetime.datetime(1997,3,1)

omn_dbdir = "../data/sqlite3/"
omn_db_name = "omni_sw_imf.sqlite"
omn_table_name = "imf_sw"
omn_train = True
omn_norm_param_file = omn_dbdir + "omn_mean_std.npy"

sml_db_name = "smu_sml_sme.sqlite"
sml_table_name = "smusmlsme"
sml_norm_param_file = omn_dbdir + "sml_mean_std.npy"

In [8]:
omnObj = omn_utils.OmnData(start_date, end_date, omn_dbdir,\
               omn_db_name, omn_table_name,\
               omn_train, omn_norm_param_file, sml_train=True, sml_norm_file=sml_norm_param_file, imf_normalize=True,\
                smlDbName=sml_db_name, sml_normalize=True, smlTabName=sml_table_name, db_time_resolution=1, \
                include_omn=True, omn_train_params = [ "By", "Bz", "Bx", "Vx", "Np" ],\
                include_sml=True, sml_train_params = [ "au", "al" ] )
omnObj.omnDF.head()

normalizing the IMF data ...
mean and std values...
(By     -0.436426
Bz     -0.432334
Bx      1.211350
Vx   -411.474961
Np      8.350599
dtype: float64, By     3.513746
Bz     3.128493
Bx     3.299880
Vx    81.025832
Np     5.099947
dtype: float64)
loaded OMNI data
normalizing the IMF data ...
mean and std values...
(au     67.772072
al   -134.423500
dtype: float64, au     61.912801
al    175.175561
dtype: float64)
loaded SML data


,datetime,By,Bz,Bx,Vx,Np,au,al
0,1997-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,-0.141684,0.539022
1,1997-01-01 00:01:00,NaN,NaN,NaN,NaN,NaN,-0.060926,0.550439
2,1997-01-01 00:02:00,NaN,NaN,NaN,NaN,NaN,-0.125533,0.510479
3,1997-01-01 00:03:00,NaN,NaN,NaN,NaN,NaN,0.052137,0.373474
4,1997-01-01 00:04:00,NaN,NaN,NaN,NaN,NaN,0.262109,0.196509


In [9]:
batchObj = batch_utils.DataUtils(omn_dbdir,\
                       omn_db_name, omn_table_name,\
                       omn_train, omn_norm_param_file,\
                        onsetDataloadFile="../data/binned_data_extra_latlon.feather",\
                        sml_train=True,useSML=True,\
                        sml_norm_file=sml_norm_param_file,\
                smlDbName=sml_db_name, sml_normalize=True, smlTabName=sml_table_name,\
                include_omn=True,smlDateRange=[start_date,end_date],\
                include_sml=False, sml_train_params = [ "au", "al" ], omn_time_delay=10)

creating onset data
('Working through year-->', 1997)
                     bin_0  mlat_0  mlt_0  del_minutes data_label  multi_ss
1997-01-01 00:00:00      0    -1.0   -1.0         -1.0          S         0
1997-01-01 00:01:00      0    -1.0   -1.0         -1.0          S         0
1997-01-01 00:02:00      0    -1.0   -1.0         -1.0          S         0
1997-01-01 00:03:00      0    -1.0   -1.0         -1.0          S         0
1997-01-01 00:04:00      0    -1.0   -1.0         -1.0          S         0
('original DF label counts---->', 0    71109
1    13852
Name: outBinary, dtype: int64)
Downsampling the data into bins by UT hour
('new DF label counts---->', 1    13852
0    13852
Name: outBinary, dtype: int64)
normalizing the IMF data ...
mean and std values...
(By     -0.436562
Bz     -0.432437
Bx      1.211522
Vx   -411.482423
Np      8.350180
dtype: float64, By     3.513658
Bz     3.128406
Bx     3.299825
Vx    81.027990
Np     5.100018
dtype: float64)
loaded OMNI data


In [10]:
x = time.time()
b1 = batchObj.batchDict[30]
y = time.time()
outOn = batchObj.onset_from_batch(b1, predList=["bin"])
z = time.time()
inOmn = batchObj.omn_from_batch(b1)
w = time.time()

print "batch calc--->", z-y
print "outOn calc--->", z-y
print "inOmn calc--->", w-z

batch calc---> 0.00647902488708
outOn calc---> 0.00647902488708
inOmn calc---> 0.0340240001678


/home/bharat/Documents/code/sson_pred/data_pipeline/batch_utils.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  _currOmnDF.interpolate(method='linear', limit_direction='both', inplace=True)


In [11]:
print outOn.shape
print inOmn.shape

(64, 1, 1)
(1, 121, 5)
